In [2]:
from networkx.generators.community import LFR_benchmark_graph
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import statistics


#initialization
n = 200
m = 230
tau1 = 3
tau2 = 1.5
mu = 0.3

pop1_belief = [0,0,0,0,0,0,0,0,0,0]
pop2_belief = [0,0,0,0,0,0,0,0,0,0]


#Creating Population 1
G1 = LFR_benchmark_graph(n, tau1, tau2, mu, average_degree=15, min_community=20, seed=10)
nx.set_node_attributes(G1, 0,  name="beliefspace")
nx.set_node_attributes(G1, 1, name="Population")
nx.set_node_attributes(G1, 0, name="acceptance")
# nx.set_node_attributes(G1, 0, name="value")
nx.set_edge_attributes(G1, 0, name="weight")



#Creating Population 2
G2 = LFR_benchmark_graph(m, tau1, tau2, mu, average_degree=15, min_community=40, seed=10)
nx.set_node_attributes(G2, 0 , name="beliefspace")
nx.set_node_attributes(G2, 2 , name="Population")
nx.set_node_attributes(G2, 0, name="acceptance")
# nx.set_node_attributes(G2, 0, name="value")
nx.set_edge_attributes(G2, 0, name="weight")

G1.graph['Actions'] = []
G2.graph['Actions'] = []

for i in range(1,76):
    if(i <= 25):
        (G1.graph['Actions']).append(i) 
    if( 25 < i <= 50):
        (G1.graph['Actions']).append(i) 
        (G2.graph['Actions']).append(i) 
    if ( i > 50):
        (G2.graph['Actions']).append(i) 


#set belief space for individuals in Population 1 
for i in G1.nodes:
    G1.nodes[i]['beliefspace'] = np.random.randint(100, size = 10)
    

    
#set accessibility for individuals in Population 1 
for i in G1.nodes:
    G1.nodes[i]['acceptance'] = np.random.randint(10,50)

    

#calculating normative knowledge for population 1
for node in range(n):
    pop1_belief += G1.nodes[node]['beliefspace']

pop1_belief = (np.around((pop1_belief/n), 0)).astype(int)


#attributes of population1
G1.graph['beliefspace'] = pop1_belief
G1.graph['Population'] = 1


#set belief space for individuals in Population 2 
for i in G2.nodes:
    G2.nodes[i]['beliefspace'] = np.random.randint(-80, 20, size = 10)
    
#set belief space for a community in pop2
# communities = {frozenset(G2.nodes[v]["community"]) for v in G2}

# com_list = list(communities)

# for i in com_list[0]:
#     G2.nodes[i]['beliefspace'] = np.random.randint(100, size = 10)
    

    
#set accessibility for individuals in Population 2 
for i in G2.nodes:
    G2.nodes[i]['acceptance'] = np.random.randint(10, 50)


    
#calculating normative knowledge for population2
for node in range(m):
    for item1 in range(10):
        pop2_belief[item1] = pop2_belief[item1] + G2.nodes[node]['beliefspace'][item1]
for item1 in range(10):
    pop2_belief[item1] = np.around((pop2_belief[item1]/m), 0).astype(int)



#attributes of population2
G2.graph['beliefspace'] = pop2_belief
G2.graph['Population'] = 2



#set edge weights 
Total_dis = 0


def setWeight(G):
    for i in G.nodes:
        for x in G.edges(i):
            k = 0   # k = coefficient
            distance = 0 # abs(difference) of elements
            Total_dis = 0
            for j in range(10):
                distance = abs(G.nodes[i]['beliefspace'][j] - G.nodes[x[1]]['beliefspace'][j])
                if distance != 0:
                    Total_dis += distance
                    k = k+1
                else:
                    continue
            G.edges[i, x[1]]["weight"] = (k*(Total_dis/100))/100
            
        
setWeight(G1)    
setWeight(G2)



#G1 weights and sd
list_of_weights1 = []
pop1_weight = 0


for i in G1.nodes:
    for j in G1.neighbors(i):
        pop1_weight += G1.get_edge_data(i, j).get('weight')
        list_of_weights1.append(G1.get_edge_data(i, j).get('weight'))
    
pop1_weight = (pop1_weight/G1.number_of_edges())


G1.graph['weight'] = pop1_weight
G1.graph['sd'] = statistics.stdev(list_of_weights1)




#G2 weights and sd
list_of_weights2 = []
pop2_weight = 0


for i in G2.nodes:
    for j in G2.neighbors(i):
        pop2_weight += G2.get_edge_data(i, j).get('weight')
        list_of_weights2.append(G2.get_edge_data(i, j).get('weight'))
    
pop2_weight = (pop2_weight/G2.number_of_edges())



G2.graph['weight'] = pop2_weight
G2.graph['sd'] = statistics.stdev(list_of_weights2)





#writing node attributes of Population 1 
with open("project/node_attr1.txt", "w") as f:
    for i in G1.nodes:
        f.write(str(i) + ":" + str(G1.nodes[i]['beliefspace']) + ":" + str(G1.nodes[i]['acceptance']) + "\n")
        
#writing graph 1 structure
nx.write_edgelist(G1, "project/graph1.edgelist", data=["weight"])   

#writing graph 1 attribute  
with open("project/graph1_attr.txt", "w") as f:
        f.write(str(G1.graph['beliefspace']) + "\n")
        f.write(str(G1.graph['weight']) + "\n")
        f.write(str(G1.graph['sd']) + "\n")
        f.write(str(G1.graph['Actions']) + "\n")

        
        
        
#writing node attributes of Population 2 
with open("project/node_attr2.txt", "w") as f:
    for i in G2.nodes:
        f.write(str(i) + ":" + str(G2.nodes[i]['beliefspace']) + ":" + str(G2.nodes[i]['acceptance']) + "\n")
        
#writing graph 2 structure
nx.write_edgelist(G2, "project/graph2.edgelist", data=["weight"])    

#writing graph 2 attribute  
with open("project/graph2_attr.txt", "w") as f:
        f.write(str(G2.graph['beliefspace']) + "\n")
        f.write(str(G2.graph['weight']) + "\n")
        f.write(str(G2.graph['sd']) + "\n")
        f.write(str(G2.graph['Actions']) + "\n")

#print(com_list)        
        
# for i in G2.nodes:
#     print("node", i, ": " , G2.nodes[i]['beliefspace'])
    
print(G1.nodes[0])  
print(G2.graph)  


{'community': {0, 130, 3, 134, 10, 14, 15, 144, 18, 19, 149, 22, 151, 23, 26, 27, 156, 157, 32, 29, 35, 168, 40, 173, 47, 49, 50, 179, 52, 51, 182, 53, 185, 195, 75, 81, 99, 100, 104, 109, 115, 117, 121, 124, 126}, 'beliefspace': array([54,  5, 64, 94, 74, 29, 40, 63, 27, 81]), 'Population': 1, 'acceptance': 44}
{'Actions': [26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75], 'beliefspace': [-30, -29, -33, -30, -30, -31, -28, -32, -31, -31], 'Population': 2, 'weight': 0.6548215887850479, 'sd': 0.07740583692226956}


In [3]:
#creating action table1  

import numpy as np

#table for pop1
rows1, cols1 = (50, 10)
pop1_table = [["a"+str(j),[0 for i in range(cols1)],0] for j in range(rows1)]


#initialize table1

start_point = 0
end_point = 8
sum2 = 0

for i in range(rows1):
    x = np.random.randint(1,9) # min num of elements
    for j in range(x):
            y = np.random.randint(1,10) # which element ?
            pop1_table[i][1][y] = np.random.randint(start_point, end_point)
    if ( 26 <= i < 50):
        pop1_table[i][2] = np.random.randint(20, 50)
    else :
        pop1_table[i][2] = np.random.randint(-100, 100)
    sum2 += pop1_table[i][2]       





    
print(pop1_table)
print(sum2)


[['a0', [0, 0, 0, 0, 0, 0, 0, 5, 7, 0], 25], ['a1', [0, 0, 0, 0, 6, 0, 5, 6, 6, 0], -83], ['a2', [0, 0, 0, 0, 0, 0, 1, 6, 0, 0], 94], ['a3', [0, 0, 0, 0, 0, 4, 0, 5, 0, 0], 26], ['a4', [0, 0, 0, 0, 0, 7, 0, 0, 0, 0], -96], ['a5', [0, 0, 6, 0, 3, 0, 0, 5, 7, 0], 97], ['a6', [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], -53], ['a7', [0, 0, 0, 7, 0, 0, 0, 0, 6, 0], -81], ['a8', [0, 1, 0, 0, 0, 4, 6, 0, 0, 0], 24], ['a9', [0, 3, 6, 0, 0, 4, 0, 0, 0, 4], -62], ['a10', [0, 0, 3, 3, 0, 0, 0, 0, 0, 0], -28], ['a11', [0, 4, 7, 0, 0, 6, 5, 0, 0, 0], 38], ['a12', [0, 0, 0, 4, 0, 7, 5, 7, 7, 6], -63], ['a13', [0, 2, 0, 0, 2, 0, 0, 0, 0, 3], 43], ['a14', [0, 0, 2, 0, 0, 0, 0, 5, 0, 0], -89], ['a15', [0, 5, 0, 2, 0, 6, 0, 7, 7, 7], 61], ['a16', [0, 0, 0, 0, 0, 4, 6, 0, 0, 0], 44], ['a17', [0, 6, 0, 4, 0, 0, 0, 5, 6, 0], -59], ['a18', [0, 0, 0, 7, 1, 2, 0, 0, 0, 3], 57], ['a19', [0, 0, 0, 0, 0, 0, 0, 0, 7, 0], -60], ['a20', [0, 2, 0, 0, 0, 0, 0, 3, 3, 0], -93], ['a21', [0, 0, 4, 0, 0, 0, 0, 0, 3, 5], -100], ['a22'

In [4]:
def takeFourth(elem):
        return elem[3]
    
def takeFirst(elem):
        return elem[0]

In [5]:
# creating action table2

#table for pop2
rows2, cols2 = (50, 10)
pop2_table = [["a"+str(j),[100 for i in range(cols2)], 0] for j in range(26,76)]

# print(pop2_table)

start_point = -60
end_point = 20
# c = 0

num_min_opinion = 20


for r in range(30):
    i = np.random.randint(10,50)
    
# for i in range(30):
    x = np.random.randint(1,9) # min num of elements
    for j in range(x):
            y = np.random.randint(1,10) # which element ?
            pop2_table[i][1][y] = np.random.randint(start_point, end_point)
            
#     if( 0 <= i <= 26):
#         pop2_table[i][2] = np.random.randint(-50, -20)
#     else:
#         pop2_table[i][2] = np.random.randint(-100, 100)
        
# print(pop2_table)
start_point1 = 0
end_point1 = 20        

for j in pop2_table:
    
#     print(j)
    index = pop2_table.index(j)
    count = 0
#     print(index)
    if(j[0] > "a31"):
        for el in j[1]:
            if( el == 100):
                count += 1
        if( count == 10):
            x = np.random.randint(1,9) # min num of elements
            for k in range(x):
                    y = np.random.randint(1,10) # which element ?
                    pop2_table[index][1][y] = np.random.randint(start_point1, end_point1)
                
# print(pop2_table)

list_for_values = []



for i in pop2_table:
    sum_of_elements = 0
    for j in i[1]:
        sum_of_elements += j
    
    i.append(sum_of_elements)
    
empty_list = []
    
sorted_table2 = pop2_table.sort(key=takeFourth, reverse=True)



c = 0
min2 = 0
max2 = 4

while( c < 50):
    for j in pop2_table:
        if (c == 0 ):
            j[2] = 0
    #         print(j)
        else:
            value_flag = ""
            while( value_flag != "Y"):
                accepted_value = np.random.randint(min2, max2)
#                 print("Hello")
#                 print(accepted_value)
                if(accepted_value >= pop2_table[c-1][2] and accepted_value <= 100):
                    j[2] = accepted_value
                    value_flag = "Y"
        min2 += 2
        max2 += 2
        c += 1

# c = 0
# for j in sorted_table2:
#     if ( c == 0):
#         j[2] == -100
#     while( accepted_value < )
#     accepted_value = np.random.randint(-100, 100)
#     if( )
# print(pop2_table[49])
# value_flag = ""    
# c = 1
# while( c < 50):
#     if(c>0):
#         for j in pop2_table[0:c]:
#             if(pop2_table[c][3] < j[3])
#         if(pop2_table[c][3] < pop2_table[c-1][3]):
#             value_flag == "N"
#             while( value_flag != "Y"):
#                 accepted_value = np.random.randint(-100, 100)
#                 if( accepted_value > pop2_table[c-1][2]):
#                     pop2_table[c][2] = accepted_value
#                     value_flag = "Y"
#         c += 1

# again = pop2_table.sort(key=takeFirst)

num_of_neg_values = np.random.randint(0, 20)

for i in range(num_of_neg_values):
    selected_index = np.random.randint(0,50)
    pop2_table[selected_index][2] = -pop2_table[selected_index][2]
    

for i in pop2_table:
#     i.pop(3)
    print(i)
    
# print(list_for_values)
# print(k)    
# for i in pop2_table:
#     print(i)

['a26', [100, 100, 100, 100, 100, 100, 100, 100, 100, 100], 0, 1000]
['a27', [100, 100, 100, 100, 100, 100, 100, 100, 100, 100], 4, 1000]
['a28', [100, 100, 100, 100, 100, 100, 100, 100, 100, 100], 4, 1000]
['a29', [100, 100, 100, 100, 100, 100, 100, 100, 100, 100], 8, 1000]
['a30', [100, 100, 100, 100, 100, 100, 100, 100, 100, 100], 9, 1000]
['a31', [100, 100, 100, 100, 100, 100, 100, 100, 100, 100], 10, 1000]
['a75', [100, 100, 100, 100, 100, 100, 100, 100, 100, 17], 14, 917]
['a50', [100, 100, 100, 100, 100, 100, 100, 9, 100, 100], 16, 909]
['a70', [100, 100, 100, 100, 100, 100, 4, 100, 100, 100], 19, 904]
['a72', [100, 100, 100, 100, 100, 100, 3, 100, 100, 100], 19, 903]
['a68', [100, 100, 100, 100, 100, 100, 100, -8, 100, 100], 20, 892]
['a59', [100, 100, 100, 100, 100, 100, 100, -56, 100, 100], 23, 844]
['a66', [100, 19, 100, 100, 100, 100, 100, 100, 100, 16], 25, 835]
['a46', [100, 100, 13, 100, 18, 100, 100, 100, 100, 100], 28, 831]
['a38', [100, 100, 18, 100, 11, 100, 100, 100

In [6]:
def List_of_actions(n):
#     print(n)
    Acts = []
    for i in n:
        for j in i:
                Acts.append(j[0])
    return(Acts)

In [7]:

def Observation_Function(iterations, migrant_node, para):   #another parameter, 0 = observation, 1 = action selection
    
    
    
    if( para == 1 ): # action selection
        iterations == 1
    
    
    action_selection = []
    neighbour_num_actions = 0
    neighbours_eligible_actions = 0
    all_eligible_actions = []

    #empty action table2 for the migrant
#     G2.nodes[migrant_node]['action_table2'] = []
    
    #start of observation 
    
    #empty value attribute for all nodes
    for it in range(iterations):
        for i in G2.nodes:
            G2.nodes[i]['value'] = 0

        list_of_min_opinion = []
        ind_action_selection = []
        action_selection=[]
        
        #returning 20 possible actions in each iteration
        list_of_min_opinion = random_task(20)
        

        for i in G2.nodes:    #func(nodes)

            acceptable_actions = []
            ind_action_selection = []
            if( i != migrated_individual):
                for action in list_of_min_opinion:  # check minimum requirement
                    min_opinion_flag = 0
                    for index in range(10): 
                        if( G2.nodes[i]['beliefspace'][index] <= action[1][index] ):
                            min_opinion_flag = min_opinion_flag + 1
                    if( min_opinion_flag == 10 ):                   
                         acceptable_actions.append([action[0], action[1], action[2]])
                all_eligible_actions.append([i,acceptable_actions])
#                 print(all_eligible_actions[0][0])
                acceptable_actions.sort(key=takeThird, reverse=True)  # sort 5 best actions with higher rewards
                best_actions = []
#                 if ( i == 30):
#                     print(acceptable_actions[0:5])
                for k in acceptable_actions[0:5]:   # adding values to value attribute
                    if(k != []):
                        action_selection.append([i,k])
                        G2.nodes[i]['value'] += k[2]
#                 if( i == 30):
#                     print("Node 30 actions" , action_selection)
#                     for act in action_selection:
#                         if(act[0] == 30):
#                             print(act)  
            elif ( para == 1):    # action selection for migrant node
                mig_acc_actions = func2(list_of_min_opinion)
                

        for i in G2[migrant_node]: # search in the neighbor list of the migrated node
            for j in action_selection: # start the search
                 if ( i == j[0] and j[1] != []):   # if the node is a neighbor node
                    if (j[1][0] not in List_of_actions(G2.nodes[migrant_node]['action_table2'])): #if action doesnt already exist in table2 of the migrant node
                            G2.nodes[migrant_node]['action_table2'].append([j[1]])  #append it
                    
        
        
    
    for mig_neighbour in G2[migrant_node]:
        for o in all_eligible_actions:
            if( mig_neighbour == o[0]):
                neighbours_eligible_actions += len(o[1])
#                 print(o[1])
                
    
    neighbours_eligible_actions = neighbours_eligible_actions/len(G2[migrant_node])
    
    
    print(neighbours_eligible_actions)
#     for i in G2[migrant_node]: # search in the neighbor list of the migrated node
#             for j in acceptable_actions: # start the search
#                 if(i == j[0] and j[1] != []):
#                     print(j)
                    
    if ( para == 1):
        return([G2.nodes[migrant_node]['action_table2'], mig_acc_actions, neighbours_eligible_actions])
    if ( para == 0):
        return([G2.nodes[migrant_node]['action_table2'], neighbours_eligible_actions])


In [128]:
print(Observation_Function(3, migrated_individual, 0))


    

KeyError: 'action_table'

In [8]:
def random_task(r):    # returning list of minimum opinion of 20 available actions in each iteration
    list_of_min_opinion = []
    
    possible_actions = sample(G2.graph['Actions'], r)
    for actions in possible_actions:
                for k in G2.graph['action_table']:
                    if( actions == k[0]):
                        list_of_min_opinion.append([k[0],k[1],k[2]])

    return(list_of_min_opinion)

In [11]:

# def func2(l2):
    
    

#         migrant_actions2 = []
#         migrant_actions1 = []
#         migrant_actions = []
#     # print(G2.nodes[migrated_individual]['action_table2'])


#     for j in G2.nodes[migrated_individual]['action_table2']:
#             for p in j:
#                  if p[0] not in migrant_actions2:
#                         migrant_actions2.append(p[0])

#         # # print(migrant_actions2)

#     for j in G2.nodes[migrated_individual]['action_table1']:
#                  if j[0] not in migrant_actions1:
#                         migrant_actions1.append([j[0], j[1], j[2]])

#     # print("m action 1", migrant_actions1) #i[2]
#     # print(migrant_actions2) #



    

#     # print("L2", l2)

#     for i in l2:


#             if i[0] in migrant_actions2: 

#                 migrant_actions.append([i[0], i[1], i[2]])

#             else:
#                 for m in migrant_actions1:
#                     if( i[0] == m[0]):
#     #                     print("Hello", m[0], m[1]) 
#                 migrant_actions.append([i[0], i[1], m[2]])


#     acceptable_actions = []
#     ind_action_selection = []

#     for action in migrant_actions:
#         #                 print(list_of_min_opinion)
#                         min_opinion_flag = 0
#                         for index in range(10):
#         #                     print(action[1][index])
#                             if( G2.nodes[migrated_individual]['beliefspace'][index] <= action[1][index] ):
#                                 min_opinion_flag = min_opinion_flag + 1

#         #break                        
#         #                    if( G2.nodes[i]['beliefspace'][index] > action[1][index] ):
#         #                        min_opinion_flag = min_opinion_flag - 1

#         #                     print(min_opinion_flag)
#                         if( min_opinion_flag == 10 ):
#         #                      print('y')
#                              acceptable_actions.append([action[0], action[1], action[2]])
#         #                      print(acceptable_actions)
#         #                 if( min_opinion_flag != 10 ):
#         #             print(i)           
#         #             print(acceptable_actions)
#                         acceptable_actions.sort(key=takeThird, reverse=True)
#             #             print(acceptable_actions)
#             #             best_actions = acceptable_actions[:5]    
#                         best_actions = []
#                         for k in acceptable_actions[0:5]:
#                             if(k != []):
#                                 #best_actions.append(k)
#         #                         print(k[2])
#                                 action_selection.append([i,k])

#         #            action_selection.append([i, best_actions])

#                         for act in action_selection:
#                             for p in l2:
#                                 if( act[0] == p[0]):
#                                     G2.nodes[migrated_individual]['value'] += p[2]
#                                     break
                                
    
#     for j in action_selection: # start the search
#                   # if the node is an neighbor node
#                     print(j)
#                     if (j[1][0] not in List_of_actions(G2.nodes[migrant_node]['action_table2'] and j[1] != [])):
#                             G2.nodes[migrant_node]['action_table2'].append([j[1]])                            
    
#     return(0)


In [9]:
#action selection of migrated_individual using its knowledge history while learning

def func2(l2):
    

    migrant_actions2 = []
    migrant_actions1 = []
    migrant_actions = []
    
    action_selection1 = []
    
    
    #create a list of actions in action table 2 
    for j in G2.nodes[migrated_individual]['action_table2']:
            for p in j:
                 if p[0] not in migrant_actions2:
                        migrant_actions2.append(p[0])

    #create a list of actions in action table 1
    for j in G2.nodes[migrated_individual]['action_table1']:
                 if j[0] not in migrant_actions1:
                        migrant_actions1.append([j[0], j[1], j[2]])
                    
    for i in l2:
            if i[0] in migrant_actions2: #if action i exists in action table 2 append it and its value to migrant actions
                migrant_actions.append([i[0], i[1], i[2]])
                

            else:
                for m in migrant_actions1: #if action i exists in action table 1 append it and the value of it in table 1 to migrant actions
                    if( i[0] == m[0]):
                        migrant_actions.append([i[0], i[1], m[2]])
                        


    acceptable_actions = []
    ind_action_selection = []
    
    
    
#     print(migrant_actions,"--------------------")
#     print(G2.nodes[migrated_individual]['beliefspace'])
    
    for action in migrant_actions:   # check  the minimum requirement for actions in migrant_actions
                        
#                         print(action[1])
                        min_opinion_flag = 0
                        for index in range(10):
#                             print(G2.nodes[migrated_individual]['beliefspace'][index], "-------",action[1][index])
                            if( G2.nodes[migrated_individual]['beliefspace'][index] <= action[1][index] ):
                                min_opinion_flag = min_opinion_flag + 1
#                         print(action, "------------" , min_opinion_flag)
                        if( min_opinion_flag == 10 ):
#                             print("Yeah")
                            acceptable_actions.append([action[0], action[1], action[2]])
    acceptable_actions.sort(key=takeThird, reverse=True)   #sort them by best value
#                         print(acceptable_actions)
    best_actions = []
#                         print(action_selection1, "222222222222222222222222222222222")
    count = 0
#                         print("ACCEptable ACTIONS", acceptable_actions[0:5])
#     if ( len(acceptable_actions)  > 5):
#         for c in range(5):
#             if ( acceptable_actions[c] != []):
#                 action_selection1.append([migrated_individual, acceptable_actions[c]])
                
#     else:
#         for c in range(len(acceptable_actions)):
#                     if ( acceptable_actions[c] != []):
#                             action_selection1.append([migrated_individual, acceptable_actions[c]])
                            
    for k in acceptable_actions[0:5]:  # select 5 best actions
            count = count + 1
            if(k != []):
                    action_selection1.append([migrated_individual, k])
#                         print(action_selection1, "\n")
    
#     print("Action Selection1", action_selection1, "\n")
    
#     print("List of min opinion", l2, "\n")
    
    
    for act in action_selection1:
            for p in l2:
#                     print("------------------", act[1][0] , " ", p[0])
                    if( act[1][0] == p[0]):   #if action exists in available actions
#                             print("Yeeeeeeeeeeeeees")
                            G2.nodes[migrated_individual]['value'] += p[2]   # add its value to value attribute 
                            break
                                
#     print("------------------",action_selection1, "-------------------------------")
    for j in action_selection1: # add 5 performed actions to table 2 of migrant 
#                     print(j)
                    if (j[1][0] not in List_of_actions(G2.nodes[migrated_individual]['action_table2']) and j[1] != []):
                            G2.nodes[migrated_individual]['action_table2'].append([j[1]])                            
#     print(len(acceptable_actions))
#     print(acceptable_actions)
    return(len(acceptable_actions))



In [1]:
# print(Observation_Function(3, migrated_individual, 1))

# print(G2.nodes[migrated_individual])

# print(G2.nodes[30])



In [12]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import statistics
from random import choice
from random import sample
from shutil import copyfile
import sys
import pandas as pd
import os 
import xlsxwriter


# initial setup


df_distance_ave = pd.DataFrame()
df_value = pd.DataFrame()
df_eligible_actions = pd.DataFrame()
df_original_opinion = pd.DataFrame()
it_new = []

for experiment in range (2):
    count_first=0
    iteration_first=0
    # Reading graph 1 structure
    G1 =nx.read_weighted_edgelist("project/graph1.edgelist", create_using = nx.Graph(), nodetype = int)
    # Reading graph 2 structure
    G2 =nx.read_weighted_edgelist("project/graph2.edgelist", create_using = nx.Graph(), nodetype = int)

    
    
    
    # Reading node attributes of Population 1
    with open("project/node_attr1.txt", "r") as f:
        lines = f.readlines()
        for line in lines:
            a = []
            line = line.split(':')
            node = int(line[0])
            belief1 = line[1]
            acceptance= int(line[2])/100
#             value = int(line[3])
            removed = ''
            for j in belief1:
                if(j == '[' or j == ']'):
                    continue
                else:
                    removed += j
                new = removed.split()
            for j in new:
                x = int(j)
                a.append(x)
            results = np.array(a)
            attrs = {
                'beliefspace' : results,
                'population' : 1,
                'distance_in' : 0,
                'distance_out' : 0,
                'acceptance': acceptance,
#                 'value' : value
            }
            G1.add_node(node, **attrs)
            
            
    
           
    # Reading graph 1 attribute
    with open("project/graph1_attr.txt") as fi:
        lines = fi.readlines()
        belief1 = lines[0]
        new = []
        a = []
        removed2 = ''
        for j in belief1:
            if(j == '[' or j == ']'):
                continue
            else:
                removed2 += j
            new = removed2.split()
        for j in new:
            x = int(j)
            a.append(x)
        results2 = np.array(a)
        G1.graph['beliefspace'] = results2
        g1_weight = float(lines[1])
        G1.graph['weight'] = g1_weight
        g1_sd = float(lines[2])
        G1.graph['sd'] = g1_sd
        action_set = lines[3].replace(",", "")
        new = []
        a = []
        removed2 = ''
        for j in action_set:
            if(j == '[' or j == ']'):
                continue
            else:
                removed2 += j
            new = removed2.split()
        for j in new:
            x = int(j)
            a.append("a" + str(x))
        results2 = list(a)
#         results2 = list(results2)
        G1.graph['Actions'] = results2
        
        
        
        

    # Reading node attributes of Population 2
    with open("project/node_attr2.txt", "r") as f:
        lines = f.readlines()
        for line in lines:
            a = []
            line = line.split(':')
            node = int(line[0])
            belief2 = line[1]
            acceptance = int(line[2])/100
#             value = int(line[3])
            removed = ''
            for j in belief2:
                if(j == '[' or j == ']'):
                    continue
                else:
                    removed += j
                new = removed.split()
            for j in new:
                x = int(j)
                a.append(x)
            results = np.array(a)
            attrs = {
                'beliefspace' : results,
                'population' : 2,
                'distance_in' : 0,
                'distance_out' : 0,
                'acceptance': acceptance,
#                 'value' : value
            }
            G2.add_node(node, **attrs)

    
    
    # Reading graph 2 attribute
    with open('project/graph2_attr.txt') as fi:
        lines = fi.readlines()
        belief1 = lines[0]
        removed2 = []
        belief1 = belief1.replace("[", "")
        new = belief1.replace("]", "")
        new = new.replace(",", "")
        belief1 = new.split()
        for j in belief1:
            removed2.append(int (j))
        results2 = np.array(removed2)
        G2.graph['beliefspace'] = results2
        g2_weight = float(lines[1])
        G2.graph['weight'] = g2_weight
        g2_sd = float(lines[2])
        G2.graph['sd'] = g2_sd
        action_set = lines[3].replace(",", "")
        new = []
        a = []
        removed2 = ''
        for j in action_set:
            if(j == '[' or j == ']'):
                continue
            else:
                removed2 += j
            new = removed2.split()
        for j in new:
            x = int(j)
            a.append("a" + str(x))
        results2 = list(a)
        #results2 = list(results2)
        G2.graph['Actions'] = results2
        
    
    
    nx.set_node_attributes(G1, 0, name="value")
    nx.set_node_attributes(G2, 0, name="value")
    G1.graph['action_table'] = pop1_table
    G2.graph['action_table'] = pop2_table
    
#     print(G2.nodes[0])
    
#     print(G1.graph['action_table'])
#     print(G2.graph)
    
    #functions

    def setWeight(G):
        for i in G.nodes:
            for x in G.edges(i):
                k = 0  # k = coefficient
                distance = 0  # abs(difference) of elements
                Total_dis = 0
                for j in range(10):
                    distance = abs(G.nodes[i]['beliefspace'][j] - G.nodes[x[1]]['beliefspace'][j])
                    if distance != 0:
                        Total_dis += distance
                        k = k + 1
                    else:
                        continue
                G.edges[i, x[1]]["weight"] = (k * (Total_dis / 100)) / 100
                
         
    
    def takeSecond(elem):
        return elem[1]

    def takeThird(elem):
        return elem[2]

    def sorting(n):
        for i in G1.nodes:
            if (n == G1):
                y.append([i, G1.nodes[i].get('distance_in')])
            if (n == G2):
                y.append([i, G1.nodes[i].get('distance_out')])

        y.sort(key=takeSecond)

    #Create a list of 20 individuals for scenario 3 and 4
    def create_list(numbers, rangee):
        community_list = []
        if (rangee == 0):
            for i in range(numbers):
                community_list.append(y[i][0])

            ind = np.random.choice(community_list)
            return ind
        if (rangee == 1):
            for i in range(n-20, n):
                community_list.append(y[i][0])

            ind = np.random.choice(community_list)
            return ind

    
            
    def distance_in(G):
        for i in G.nodes:
            k = 0  # k = coefficient
            distance = 0  # abs(difference) of elements
            Total_dis = 0
            for j in range(10):
                distance = abs(G.nodes[i]['beliefspace'][j] - G.graph['beliefspace'][j])
                if distance != 0:
                    Total_dis += distance
                    k = k + 1
                else:
                    continue
            G.nodes[i]['distance_in'] = k * (Total_dis / 100)
            
            
    
    def distance_out(G, G_prime):
        for i in G.nodes:
            k = 0   # k = coefficient
            distance = 0 # abs(difference) of elements
            Total_dis = 0
            for j in range(10):
                distance = abs(G.nodes[i]['beliefspace'][j] - G_prime.graph['beliefspace'][j])
                if distance != 0:
                    Total_dis += distance
                    k = k+ 1
                else:
                    continue
            G.nodes[i]['distance_out'] = k * (Total_dis / 100)

    
    def dis_btw_inds(the_list):
        list_of_dis = []
        for ind in the_list:
            k = 0  # k = coefficient
            distance = 0  # abs(difference) of elements
            Total_dis = 0
            for j in range(10):
                distance = abs(G2.nodes[migrated_individual]['beliefspace'][j] - G2.nodes[ind]['beliefspace'][j])
                if distance != 0:
                    Total_dis += distance
                    k = k + 1
            ind_dis = k * (Total_dis / 100)
            list_of_dis.append([ind, ind_dis])
        
        return list_of_dis
    

    # selecting an individual
    #scenario = int(input("Select an individual: \n 1. Elite \n 2. Random \n 3. Similar to destination Population \n 4. Isolated \n"))
    
    scenario=1
    global y
    y = []


    individual = 0
    distance_in(G1)
    distance_in(G2)
    distance_out(G1,G2)
    distance_out(G2,G1)
    setWeight(G1)
    setWeight(G2)
    
    
    
    
    if (scenario == 1):
        sorting(G1)
        individual = create_list(20, 0)
        
    if (scenario == 2):
        individual = np.random.randint(0, G1.number_of_nodes())

    if (scenario == 3):
        sorting(G2)
        individual = create_list(20, 0)
        
    if (scenario == 4):
        sorting(G1)
        individual = create_list(0, 1)
        
        
    
    
    # migration
    Belief = G1.nodes[individual]["beliefspace"]
#     val1 = G1.nodes[individual]["value"]
    Pop_flag = G1.nodes[individual]["population"]
    acceptance_1= G1.nodes[individual]["acceptance"]
    
    migrated_individual = G2.number_of_nodes()

    G2.add_node(migrated_individual, beliefspace=Belief, population=Pop_flag, acceptance= acceptance_1, distance_out=0, distance_in=0)
    G2.nodes[migrated_individual]['action_table1'] = G1.graph['action_table']
#     G2.nodes[migrated_individual]['action_table2'] = []
    G2.nodes[migrated_individual]['value'] = 0
    G2.nodes[migrated_individual]['action_table2'] = []
    
    
    original_belief = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for i in range(10):
        original_belief[i] = G2.nodes[migrated_individual]['beliefspace'][i]
    
    
#     print(G2.nodes[migrated_individual])
    
    def original_opinion():
            k = 0  # k = coefficient
            distance = 0  # abs(difference) of elements
            Total_dis = 0
            for j in range(10):
                distance = abs(G2.nodes[migrated_individual]['beliefspace'][j] - original_belief[j])
                if distance != 0:
                    Total_dis += distance
                    k = k + 1
            dis = k * (Total_dis / 10)
            return dis
        
        
    #Finding the best community to connect to
#     com1_dis = dis_btw_inds(com1_list)
# #     print(com1_dis)
#     com2_dis = dis_btw_inds(com2_list)
# #     print(com2_dis)
    
#     sum1 = 0
#     sum2 = 0
#     for i,j in com1_dis:
#         sum1 += j 
#     ave1 = (sum1/len(com1_dis))
# #     print(ave1)
    
#     for i,j in com2_dis:
#         sum2 += j 
#     ave2 = (sum2/len(com2_dis))
# #     print(ave2)
    
    
        
    # option 1 : Random
    
    selected_ind = np.random.randint(0,(G2.number_of_nodes()-1))
    G2.add_edge(migrated_individual, selected_ind)
    setWeight(G2)
    
    
    # option 2 : connect to the most similar person
    
    #x_list = np.random.randint(0, G2.number_of_nodes() - 1, 23)
    #list_of_dis = []
    #list_of_dis = dis_btw_inds(x_list)
    #list_of_dis.sort(key=takeSecond)
    #selected_ind = list_of_dis[0][0]
    #G2.add_edge(migrated_individual, selected_ind)
    #setWeight(G2)
    
    # option 3 : connect to a community
#     if( ave1 < ave2):
#         for i in range(3):
#             selected_ind = np.random.choice(com1_list)
# #             G2.add_edge(migrated_individual, selected_ind)
# #             print(selected_ind)
        
#     else:
#         for i in range(3):
#             selected_ind = np.random.choice(com2_list)
# #             G2.add_edge(migrated_individual, selected_ind)
# #             print(selected_ind)
        
#     G2.add_edge(migrated_individual, selected_ind)
#     setWeight(G2)


    
    
    # copying previous files
    file_name = str(experiment) 
    file_address="project/copy/files/" 
    
    copyfile("project/node_attr1.txt", file_address + file_name + "_node_attr1_copy.txt")
    copyfile("project/graph1.edgelist", file_address + file_name + "_graph1_copy.edgelist")
    copyfile("project/graph1_attr.txt", file_address + file_name + "_graph1_attr_copy.txt")

    copyfile("project/node_attr2.txt", file_address + file_name + "_node_attr2_copy.txt")
    copyfile("project/graph2.edgelist", file_address + file_name + "_graph2_copy.edgelist")
    copyfile("project/graph2_attr.txt", file_address + file_name + "_graph2_attr_copy.txt")

    # adding new edge to file
    new_node_to_string = str(migrated_individual)
    selected_to_string = str(selected_ind)
    weights = str(G2.edges[migrated_individual, selected_ind]['weight'])

    with open(file_address + file_name + "_graph2_copy.edgelist", 'a') as f:
        f.write(new_node_to_string + " " + selected_to_string + " " + weights + "\n")

    # adding new node attribute to file
    with open(file_address + file_name + "_node_attr2_copy.txt", 'a') as f:
        f.write("--------------------migration------------------" + "\n")
        f.write(new_node_to_string + ":" + str(G2.nodes[migrated_individual]['beliefspace']) + "\n")


    
    distance_in(G1)
    distance_in(G2)
    distance_out(G1,G2)
    distance_out(G2,G1)
    
    
    #creating a list of nodes for comparison
    major_node = migrated_individual
    selected = selected_ind
    nodes = [major_node, selected]
    
    while( len(nodes) < 22 ):
        node = np.random.randint(0, G2.number_of_nodes()-1)
        if( node != selected ):
            nodes.append(node)
            
    #original_beliefs = []
    #for node in nodes:
    #    original_beliefs.append((node, G2.nodes[node]['beliefspace']))
            
    

    
    
    
    G2_belief = G2.graph['beliefspace']
    
    df_list = []
    df_final = pd.DataFrame()
    df_final2 = pd.DataFrame()
    final_list = []
    migrant_dis = []
    rest_dis = []
    final_sd = []
    original = []
    network_value = []
    neighbours_value = []
    migrant_value = []
    done = 0
    migrant_actions2 = []
    neighbours_actions2 = []
    
    
    df_weight = pd.DataFrame(columns=['iteration', 'node', 'average_weight', 'sd'])
    df_distance = pd.DataFrame(columns=nodes)
    df_friends = pd.DataFrame(columns=nodes)
#     df_value = pd.DataFrame(columns=['network', 'migrant', 'neighbours'])
    
    
#     print()
    
    #Start of Observation
    list_of_outputs_observation = 0 
    list_of_outputs_observation = Observation_Function(3, migrated_individual, 0)
    G2.nodes[migrated_individual]['action_table2'] = list_of_outputs_observation[0]
    neighbours_actions = list_of_outputs_observation[1]
    
#     print(G2.nodes[migrated_individual], "\n")
#     print("----------", G2.nodes[29])
    
    
    
    
    
    
    # start of iterations
    # i = 1
    for iteration in range(50):
        
        pop2_belief = [0,0,0,0,0,0,0,0,0,0]
        list_of_friends = []
        fr = []
        z = 0
        df_col0 = []
        df_col1 = []
        df_col2 = []
        df_col3 = []
        df_final = pd.DataFrame()
        df = pd.DataFrame()
       
        

        
        # belief spaces change
        for i in G2.nodes:
            acceptance_rate = G2.nodes[i]['acceptance']
            talk_prob = np.random.rand()
            for j in G2.neighbors(i):
                number_of_friends = 0
                sum_of_friends = 0
                if (talk_prob <= G2.get_edge_data(i, j).get('weight')):
                    talk = 1
                else:
                    talk = 0
                if (talk == 1):
                    item = np.random.randint(0, 10)
                    if (np.random.rand()<= acceptance_rate):
                            Norm_graph2= G2.graph['beliefspace'][item] * (1 - (G2.nodes[i]['distance_in'] / 100))
                            Friend_belief= G2.nodes[j]['beliefspace'][item] * (1 - G2.get_edge_data(i, j).get('weight'))
                            opinion= G2.nodes[i]['beliefspace'][item]
                            if (i==migrated_individual): 
                                Norm_graph1 = G1.graph['beliefspace'][item] * (1 - (G2.nodes[i]['distance_out'] / 100))
                                G2.nodes[i]['beliefspace'][item] = (((1- acceptance_rate) * opinion) + ((acceptance_rate) * ((Norm_graph1 + ((Norm_graph2 + Friend_belief)/2))/2)))
                            else:
                                G2.nodes[i]['beliefspace'][item] = (((1- acceptance_rate) * opinion) + ((acceptance_rate) * ((Norm_graph2 + Friend_belief)/2)))
                       
  
                            
   

        # G2 changes
        for node in range(G2.number_of_nodes()):
            for item1 in range(10):
                pop2_belief[item1] = pop2_belief[item1] + G2.nodes[node]['beliefspace'][item1]
        for item1 in range(10):
            pop2_belief[item1] = np.around((pop2_belief[item1] / G2.number_of_nodes()), 0).astype(int)
        G2.graph['beliefspace'] = pop2_belief
        

        # distance_in and out change
        distance_in(G2)
        distance_out(G2,G1)
        

        # G2_weight update
        setWeight(G2)

        # G2 weights and sd
        pop2_weight = 0
        list_of_weights2 = []

        for i in G2.nodes:
            for j in G2.neighbors(i):
                pop2_weight += G2.get_edge_data(i, j).get('weight')
                list_of_weights2.append(G2.get_edge_data(i, j).get('weight'))

        pop2_weight = (pop2_weight / G2.number_of_edges())
        G2.graph['weight'] = pop2_weight
        G2.graph['sd'] = statistics.stdev(list_of_weights2)

        
        
        
        list_of_outputs_actions = Observation_Function(3, migrated_individual, 1)
        G2.nodes[migrated_individual]['action_table2'] = list_of_outputs_actions[0]
        mig_eligible_actions = list_of_outputs_actions[1]
        neighbours_actions = list_of_outputs_actions[2]
        
#         print(G2.nodes[migrated_individual] , "\n" , G2.nodes[30])
        
        migrant_actions2.append(mig_eligible_actions)
        neighbours_actions2.append(neighbours_actions)
        
        # finding new friends
        
        for x in G2.neighbors(migrated_individual):
            for friend in G2.neighbors(x):
                if (friend != migrated_individual and friend not in fr):
                    k2 = 0  # k = coefficient
                    distance2 = 0  # abs(difference) of elements
                    friends_dis = 0
                    dis_friends_of_friends = 0
                    for j in range(10):
                        distance2 = abs(G2.nodes[migrated_individual]['beliefspace'][j] - G2.nodes[friend]['beliefspace'][j])
                        if distance2 != 0:
                            friends_dis += distance2
                            k2 = (k2) + 1
                    weight_friends_of_friends = ((k2) * (friends_dis / 100)) / 100
                    #compare the value of migrated node with its neighbors
                    average_value_of_neighbours = 0
                    for migrant_neighbour in G2[migrated_individual]:
                        average_value_of_neighbours += G2.nodes[migrant_neighbour]['value'] 
                    if( G2.nodes[migrated_individual]['value'] > average_value_of_neighbours/2):
                        if (weight_friends_of_friends <= G2.edges[migrated_individual, x]['weight']):
                            if (np.random.rand() < G2.edges[migrated_individual, x]['weight']):
                                another = (friend, weight_friends_of_friends)
                                another = list(another)
                                list_of_friends.append(another)
                                fr.append(friend)
        
        
        
        
        
        # adding new friends
        if len(list_of_friends) != 0 :
            selected_friend= np.random.randint(0, len(list_of_friends))
            G2.add_edge(migrated_individual, list_of_friends[selected_friend][0], weight=list_of_friends[selected_friend][1])
                
                
        
        #appending new edgelist (weights)
        with open(file_address + file_name + "_graph2_copy.edgelist", 'a') as f:
            f.write("-----------------iteration" + str(iteration) + "---------------------" + "\n")
            for node in G2.nodes:
                for neighbor in G2.neighbors(node):
                    w = G2.get_edge_data(node, neighbor).get('weight')
                    f.write(str(node) + " " + str(neighbor) + " " + str(w) + "\n" )
        
        
        #appending new attributes
        with open(file_address + file_name + "_node_attr2_copy.txt", "a") as f:
            f.write("-----------------iteration" + str(iteration) + "---------------------" + "\n")
            for i in G2.nodes:
                f.write(str(i) + ":" + str(G2.nodes[i]['beliefspace']) + "\n")

        
        
        
        # appending random node attributes, migrant attribute and G2 belief space after change
        with open(file_address + file_name + ".txt", 'a') as f:
                    f.write("-----------------iteration" + str(iteration + 1) + "---------------------" + "\n \n")
                    for node in nodes:
                        if (node == migrated_individual):
                            text = str(node) + " : " + str(G2.nodes[node]['beliefspace']) + " , " + "distance_in : " + str(
                                G2.nodes[node]['distance_in']) + "\n \n" + str(list_of_friends) + "\n \n"
                        else:
                            text = str(node) + " : " + str(G2.nodes[node]['beliefspace']) + " , " + "distance_in : " + str(
                                G2.nodes[node]['distance_in']) + "\n \n"
                        f.write(text)
                    f.write("G2_belief space" + str(G2.graph['beliefspace']) + " , " + "AVE-weight : " + str(
                        G2.graph['weight']) + " , " + "sd : " + str(G2.graph['sd']) + "\n \n")
        
        
        
        #scenario1
        with open(file_address  + file_name + "_new_friend.txt", 'a') as f:
            #print(list(G2.neighbors(migrated_individual)))
            #print(len(list(G2.neighbors(migrated_individual))))
            if(done == 0): 
                if(len(list(G2.neighbors(migrated_individual))) ==  2):
                    it_new.append(iteration+1)
                    done = 1
                if(iteration == 49 and len(list(G2.neighbors(migrated_individual))) ==  1):
                    it_new.append(0)
                    
                    
        
        
        
        #scenario2
        with open(file_address + file_name + "_weight.txt", 'a') as f:
            f.write("-----------------iteration" + str(iteration + 1) + "---------------------" + "\n \n")
            for node in nodes:
                node_weight_info = []
                weight_list = []
                total_weight = 0
                for neighbor in G2.neighbors(node):
                    f.write(str(node) + "," + str(neighbor) + " : " + str(G2.edges[node,neighbor]['weight']) + "\n")
                    total_weight += G2.edges[node,neighbor]['weight']
                    weight_list.append(G2.edges[node,neighbor]['weight'])
                ave_weight = total_weight/len(list(G2.neighbors(node)))
                if( len(weight_list) > 1):
                    sd = statistics.stdev(weight_list)
                else:
                    sd = 0
                node_weight_info = [iteration, node, ave_weight, sd]
                df_weight.loc[len(df_weight), :] =  node_weight_info
                         
            f.write("\n \n")
        
        
        
        
        #scenario3
        dis_row_list = []
        
        with open(file_address + file_name + "_dis.txt", 'a') as f:
            f.write("-----------------iteration" + str(iteration + 1) + "---------------------" + "\n \n")
            for node in nodes:
                dis_row_list.append(G2.nodes[node]['distance_in'])
                f.write(str(node) + " : " + str(G2.nodes[node]['distance_in']) + "\n \n")
                
            df_distance.loc[len(df_distance), :] = dis_row_list
        
        
        
        rest_ave = 0
        standard_deviation = []
        #with open(file_address + file_name + "_dis_ave.txt", 'a') as f:
            #f.write("-----------------iteration" + str(iteration + 1) + "---------------------" + "\n \n")
        with open(file_address + file_name + "hello.txt", 'a') as f:
            for node in nodes:
                if(node == m):
                    migrant_dis.append(G2.nodes[migrated_individual]['distance_in'])
                if(node != m):
                    rest_ave += G2.nodes[node]['distance_in']
                    standard_deviation.append(G2.nodes[node]['distance_in'])
            rest_ave = rest_ave/(len(nodes) - 1)
            rest_dis.append(rest_ave)
            s_d = statistics.stdev(standard_deviation)
            final_sd.append(s_d)
            
            

        
            
        #scenario4
        new_fr_list = []
        with open(file_address + file_name + "_friends.txt", 'a') as f:
            f.write("-----------------iteration" + str(iteration + 1) + "---------------------" + "\n \n")
            for node in nodes:
                    new_fr_list.append(len(list(G2.neighbors(node))))
                    f.write(str(node) + " : " + str(len(list(G2.neighbors(node)))) + "\n \n")
            df_friends.loc[len(df_friends), :] = new_fr_list
            
           
        
        
        #scenario5
        #with open(file_address + file_name + "_original.txt", 'a') as f:
        #f.write("-----------------iteration" + str(iteration + 1) + "---------------------" + "\n \n")
        original.append(original_opinion())
        
        
        # storing values
#         network_values = []
        ave_value = 0
        ave_value_neighbours = 0
        for node in G2.nodes:
            ave_value += G2.nodes[node]['value']
            
            
        
        ave_value = ave_value/G2.number_of_nodes()
        network_value.append(ave_value)
        
        for neighbour in G2[migrated_individual]:
            ave_value_neighbours += G2.nodes[neighbour]['value']
            
        ave_value_neighbours = ave_value_neighbours/len(G2[migrated_individual])
        neighbours_value.append(ave_value_neighbours)
        
        migrant_value.append(G2.nodes[migrated_individual]['value'])
        
        
        
        
        
        
        
#         all_values = [ave_value , G2.nodes[migrated_individual]['value'], ave_value_neighbours]
        
#         df_value.loc[len(df_value), :] =  all_values
        
#         network_value = []
#         neighbours_value = []
#         migrant_value = []
        
#         print(all_values)
        
#     print("---------------------------------------------------" + "\n")
#     print(G2.nodes[migrated_individual])
#     print(all_values)
    
    #print(experiment , it_new)
    next_df_original_opinion = pd.DataFrame({ "dis_to_original": original})
    
    
            
    next_df_distance_ave = pd.DataFrame({ "migrant": migrant_dis,
                        "rest":rest_dis,
                        "sd": final_sd})
    
    next_df_value = pd.DataFrame({"network": network_value,
                                 "migrant": migrant_value,
                                 "neighbours": neighbours_value})
    
#     next_df_eligible_actions = pd.DataFrame({"migrant": mig_eligible_actions   ,
#                                             "neighbours": neighbours_actions })
    
        
    df_distance_ave = pd.concat([df_distance_ave, next_df_distance_ave], axis = 1)
    
    df_value = pd.concat([df_value, next_df_value], axis = 1)
    
#     df_eligible_actions = pd.concat([df_eligible_actions, next_df_eligible_actions], axis = 1)
    
    df_original_opinion = pd.concat([df_original_opinion, next_df_original_opinion], axis = 1)
        
    df_weight.to_csv(file_address + file_name + '_weight.csv')
    df_distance.to_csv(file_address + file_name + '_dis.csv')
    
    df_friends.to_csv(file_address + file_name + '_friends.csv')
    
#     df_value.to_csv(file_address + file_name + '_values.csv')

    print(G2.nodes[migrated_individual])
df_value.to_csv(file_address + file_name + '_value.csv')    
df_distance_ave.to_csv(file_address + file_name + '_dis_ave.csv')
df_original_opinion.to_csv(file_address + file_name + '_original.csv')
df_new_fr = pd.DataFrame({"new_connection_it": it_new })

df_new_fr.to_csv(file_address + file_name + '_new_connection.csv')




        



34.0
37.0
42.0
40.0
34.0
35.0
37.0
38.0
33.0
30.0
40.0
32.0
35.0
40.0
40.0
37.0
38.0
34.0
40.0
38.0
39.0
34.0
39.0
36.0
36.0
34.0
39.0
39.0
38.0
39.0
38.0
38.0
36.0
35.0
41.0
36.0
32.0
40.0
37.0
35.0
39.0
36.0
43.0
40.0
38.0
41.0
36.0
32.0
35.0
40.0
34.0
{'beliefspace': array([28, 27, 32, 22, 17, 58, 17, 24, 54, 46]), 'population': 1, 'acceptance': 0.46, 'distance_out': 19.099999999999998, 'distance_in': 55.199999999999996, 'action_table1': [['a0', [0, 0, 0, 0, 0, 0, 0, 5, 7, 0], 25], ['a1', [0, 0, 0, 0, 6, 0, 5, 6, 6, 0], -83], ['a2', [0, 0, 0, 0, 0, 0, 1, 6, 0, 0], 94], ['a3', [0, 0, 0, 0, 0, 4, 0, 5, 0, 0], 26], ['a4', [0, 0, 0, 0, 0, 7, 0, 0, 0, 0], -96], ['a5', [0, 0, 6, 0, 3, 0, 0, 5, 7, 0], 97], ['a6', [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], -53], ['a7', [0, 0, 0, 7, 0, 0, 0, 0, 6, 0], -81], ['a8', [0, 1, 0, 0, 0, 4, 6, 0, 0, 0], 24], ['a9', [0, 3, 6, 0, 0, 4, 0, 0, 0, 4], -62], ['a10', [0, 0, 3, 3, 0, 0, 0, 0, 0, 0], -28], ['a11', [0, 4, 7, 0, 0, 6, 5, 0, 0, 0], 38], ['a12', [0, 0, 0, 

In [21]:
len(G2[migrated_individual])


1